In [1]:
# 경고(warning) 비표시
import warnings
warnings.filterwarnings('ignore')

# 제4장 고객의 행동을 예측하는 테크닉 10

### 테크닉 31 : 데이터를 읽어들이고 확인하자

In [2]:
import pandas as pd
uselog = pd.read_csv('use_log.csv')
uselog.isnull().sum()

log_id         0
customer_id    0
usedate        0
dtype: int64

In [3]:
customer = pd.read_csv('customer_join.csv')
customer.isnull().sum()

customer_id             0
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
dtype: int64

### 테크닉 32 : 클러스터링으로 회원을 그룹화하자

In [4]:
customer_clustering = customer[["mean", "median","max", "min", "membership_period"]]
customer_clustering.head()

mean  median  max  min  membership_period
0  4.833333     5.0    8    2                 47
1  5.083333     5.0    7    3                 47
2  4.583333     5.0    6    3                 47
3  4.833333     4.5    7    2                 47
4  3.916667     4.0    6    1                 47

In [5]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
customer_clustering_sc = sc.fit_transform(customer_clustering)

kmeans = KMeans(n_clusters=4, random_state=0)
clusters = kmeans.fit(customer_clustering_sc)
customer_clustering["cluster"] = clusters.labels_
print(customer_clustering["cluster"].unique())
customer_clustering.head()

[2 1 0 3]


mean  median  max  min  membership_period  cluster
0  4.833333     5.0    8    2                 47        2
1  5.083333     5.0    7    3                 47        2
2  4.583333     5.0    6    3                 47        2
3  4.833333     4.5    7    2                 47        2
4  3.916667     4.0    6    1                 47        2

### 테크닉 33 : 클러스터링 결과를 분석하자

In [6]:
customer_clustering.columns = ["월평균값","월중앙값", "월최댓값", "월최솟값","회원기간", "cluster"]
customer_clustering.groupby("cluster").count()

월평균값  월중앙값  월최댓값  월최솟값  회원기간
cluster                              
0        1334  1334  1334  1334  1334
1         771   771   771   771   771
2        1249  1249  1249  1249  1249
3         838   838   838   838   838

In [7]:
customer_clustering.groupby("cluster").mean()

월평균값      월중앙값       월최댓값      월최솟값       회원기간
cluster                                                    
0        5.541974  5.392804   8.757871  2.704648  14.857571
1        3.065504  2.900130   4.783398  1.649805   9.276265
2        4.677561  4.670937   7.233787  2.153723  36.915933
3        8.064079  8.050716  10.014320  6.180191   7.016706

### 테크닉 34 : 클러스터링 결과를 가시화하자

In [8]:
from sklearn.decomposition import PCA
X = customer_clustering_sc
pca = PCA(n_components=2)
pca.fit(X)
x_pca = pca.transform(X)
pca_df = pd.DataFrame(x_pca)
pca_df["cluster"] = customer_clustering["cluster"]

In [9]:
import matplotlib.pyplot as plt
%matplotlib inline
for i in customer_clustering["cluster"].unique():
    tmp = pca_df.loc[pca_df["cluster"]==i]
    plt.scatter(tmp[0], tmp[1])

### 테크닉 35 : 클러스터링 결과를 바탕으로 탈퇴회원의 경향을 파악하자

In [10]:
customer_clustering = pd.concat([customer_clustering, customer], axis=1)
customer_clustering.groupby(["cluster","is_deleted"],as_index=False).count()[["cluster","is_deleted","customer_id"]]

cluster  is_deleted  customer_id
0        0           0          791
1        0           1          543
2        1           1          771
3        2           0         1231
4        2           1           18
5        3           0          820
6        3           1           18

In [11]:
customer_clustering.groupby(["cluster","routine_flg"],as_index=False).count()[["cluster","routine_flg","customer_id"]]

cluster  routine_flg  customer_id
0        0            0          227
1        0            1         1107
2        1            0          499
3        1            1          272
4        2            0            2
5        2            1         1247
6        3            0           51
7        3            1          787

### 테크닉 36 : 다음달의 이용횟수 예측을 위해 테이터를 준비하자

In [12]:
uselog["usedate"] = pd.to_datetime(uselog["usedate"])
uselog["연월"] = uselog["usedate"].dt.strftime("%Y%m")
uselog_months = uselog.groupby(["연월","customer_id"],as_index=False).count()
uselog_months.rename(columns={"log_id":"count"}, inplace=True)
del uselog_months["usedate"]
uselog_months.head()

연월 customer_id  count
0  201804    AS002855      4
1  201804    AS009013      2
2  201804    AS009373      3
3  201804    AS015315      6
4  201804    AS015739      7

In [13]:
year_months = list(uselog_months["연월"].unique())
predict_data = pd.DataFrame()
for i in range(6, len(year_months)):
    tmp = uselog_months.loc[uselog_months["연월"]==year_months[i]]
    tmp.rename(columns={"count":"count_pred"}, inplace=True)
    for j in range(1, 7):
        tmp_before = uselog_months.loc[uselog_months["연월"]==year_months[i-j]]
        del tmp_before["연월"]
        tmp_before.rename(columns={"count":"count_{}".format(j-1)}, inplace=True)
        tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    predict_data = pd.concat([predict_data, tmp], ignore_index=True)
predict_data.head()

연월 customer_id  count_pred  count_0  count_1  count_2  count_3  \
0  201810    AS002855           3      7.0      3.0      5.0      5.0   
1  201810    AS008805           2      2.0      5.0      7.0      8.0   
2  201810    AS009373           5      6.0      6.0      7.0      4.0   
3  201810    AS015233           7      9.0     11.0      5.0      7.0   
4  201810    AS015315           4      7.0      3.0      6.0      3.0   

   count_4  count_5  
0      5.0      4.0  
1      NaN      NaN  
2      4.0      3.0  
3      7.0      NaN  
4      3.0      6.0

In [14]:
predict_data = predict_data.dropna()
predict_data = predict_data.reset_index(drop=True)
predict_data.head()

연월 customer_id  count_pred  count_0  count_1  count_2  count_3  \
0  201810    AS002855           3      7.0      3.0      5.0      5.0   
1  201810    AS009373           5      6.0      6.0      7.0      4.0   
2  201810    AS015315           4      7.0      3.0      6.0      3.0   
3  201810    AS015739           5      6.0      5.0      8.0      6.0   
4  201810    AS019860           7      5.0      7.0      4.0      6.0   

   count_4  count_5  
0      5.0      4.0  
1      4.0      3.0  
2      3.0      6.0  
3      5.0      7.0  
4      8.0      6.0

### 테크닉 37 : 특징이 되는 변수를 추가하자

In [15]:
predict_data = pd.merge(predict_data, customer[["customer_id","start_date"]], on="customer_id", how="left")
predict_data.head()

연월 customer_id  count_pred  count_0  count_1  count_2  count_3  \
0  201810    AS002855           3      7.0      3.0      5.0      5.0   
1  201810    AS009373           5      6.0      6.0      7.0      4.0   
2  201810    AS015315           4      7.0      3.0      6.0      3.0   
3  201810    AS015739           5      6.0      5.0      8.0      6.0   
4  201810    AS019860           7      5.0      7.0      4.0      6.0   

   count_4  count_5  start_date  
0      5.0      4.0  2016-11-01  
1      4.0      3.0  2015-11-01  
2      3.0      6.0  2015-07-01  
3      5.0      7.0  2017-06-01  
4      8.0      6.0  2017-10-01

In [16]:
predict_data["now_date"] = pd.to_datetime(predict_data["연월"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
from dateutil.relativedelta import relativedelta
predict_data["period"] = None
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = delta.years*12 + delta.months
predict_data.head()

연월 customer_id  count_pred  count_0  count_1  count_2  count_3  \
0  201810    AS002855           3      7.0      3.0      5.0      5.0   
1  201810    AS009373           5      6.0      6.0      7.0      4.0   
2  201810    AS015315           4      7.0      3.0      6.0      3.0   
3  201810    AS015739           5      6.0      5.0      8.0      6.0   
4  201810    AS019860           7      5.0      7.0      4.0      6.0   

   count_4  count_5 start_date   now_date period  
0      5.0      4.0 2016-11-01 2018-10-01     23  
1      4.0      3.0 2015-11-01 2018-10-01     35  
2      3.0      6.0 2015-07-01 2018-10-01     39  
3      5.0      7.0 2017-06-01 2018-10-01     16  
4      8.0      6.0 2017-10-01 2018-10-01     12

### 테크닉 38 : 다음달 이용횟수를 예측하는 모델을 구축하자

In [17]:
predict_data = predict_data.loc[predict_data["start_date"]>=pd.to_datetime("20180401")]
from sklearn import linear_model
import sklearn.model_selection
model = linear_model.LinearRegression()
X = predict_data[["count_0","count_1","count_2","count_3","count_4","count_5","period"]]
y = predict_data["count_pred"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)
model.fit(X_train, y_train)

LinearRegression()

In [18]:
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.6120989361892984
0.5934730346843475


### 테크닉 39 : 모델에 기여하는 변수를 확인하자

In [19]:
coef = pd.DataFrame({"feature_names":X.columns, "coefficient":model.coef_})
coef

feature_names  coefficient
0       count_0     0.322256
1       count_1     0.200836
2       count_2     0.156345
3       count_3     0.185526
4       count_4     0.104911
5       count_5     0.050871
6        period     0.063681

### 테크닉 40 : 다음달의 이용횟수를 예측하자

In [20]:
x1 = [3, 4, 4, 6, 8, 7, 8]
x2 = [2, 2, 3, 3, 4, 6, 8]
x_pred = [x1, x2]

In [21]:
model.predict(x_pred)

array([3.85582233, 1.94845298])

In [22]:
uselog_months.to_csv("use_log_months.csv",index=False)